# Limpieza de datos

In [2]:
import pandas as pd
import numpy as np

In [3]:
predictors = ['apcp_sfc','dlwrf_sfc','dswrf_sfc','pres_msl','pwat_eatm',\
                  'spfh_2m','tcdc_eatm','tcolc_eatm','tmax_2m','tmin_2m',\
                  'tmp_2m','tmp_sfc','ulwrf_sfc','ulwrf_tatm','uswrf_sfc']
train_end = '_latlon_subset_19940101_20071231.nc'
path='data/train/'

In [4]:
from netCDF4 import Dataset
def load_data(filename):
    data=Dataset(filename)
    return data

ModuleNotFoundError: No module named 'ipynb.fs.full.utils_solar_energy_prediction'

In [ ]:
import os
apcp_sfc=load_data(os.path.join(path,predictors[0]+train_end))
dlwrf_sfc=load_data(os.path.join(path,predictors[1]+train_end))
dswrf_sfc=load_data(os.path.join(path,predictors[2]+train_end))
pres_msl=load_data(os.path.join(path,predictors[3]+train_end))
pwat_eatm=load_data(os.path.join(path,predictors[4]+train_end))
spfh_2m=load_data(os.path.join(path,predictors[5]+train_end))
tcdc_eatm=load_data(os.path.join(path,predictors[6]+train_end))
tcolc_eatm=load_data(os.path.join(path,predictors[7]+train_end))
tmax_2m=load_data(os.path.join(path,predictors[8]+train_end))
tmin_2m=load_data(os.path.join(path,predictors[9]+train_end))
tmp_2m=load_data(os.path.join(path,predictors[10]+train_end))
tmp_sfc=load_data(os.path.join(path,predictors[11]+train_end))
ulwrf_sfc=load_data(os.path.join(path,predictors[12]+train_end))
ulwrf_tatm=load_data(os.path.join(path,predictors[13]+train_end))
uswrf_sfc=load_data(os.path.join(path,predictors[14]+train_end))

In [ ]:
#Funcion que convierte a dataframe los netCDF4
from matplotlib import pyplot as plt
import pandas as pd
import xarray as xr

def net_df(num,key):
    os.chdir(r'C:\Users\MEMORY SISTEMAS\Desktop\Bootcamp\thebridge_ft_sep22 - PB\3-Machine Learning\2-Supervisado\11- Entregables')
    dp = xr.open_dataset(os.path.join(path,predictors[num]+train_end))
    dp[key]
    m2 = dp.to_dataframe()
    s = pd.DataFrame(m2)
    return(s)

In [ ]:
apcp_sfc = net_df(0,'Total_precipitation')
dlwrf_sfc = net_df(1,'Downward_Long-Wave_Rad_Flux')
dswrf_sfc = net_df(2,'Downward_Short-Wave_Rad_Flux')
pres_msl = net_df(3,'Pressure')
pwat_eatm = net_df(4,'Precipitable_water')
spfh_2m = net_df(5, 'Specific_humidity_height_above_ground')
tcdc_eatm = net_df(6,'Total_cloud_cover')
tcolc_eatm = net_df(7,'Total_Column-Integrated_Condensate')
tmax_2m = net_df(8,'Maximum_temperature')
tmin_2m = net_df(9,'Minimum_temperature')
tmp_2m = net_df(10,'Temperature_height_above_ground')
tmp_sfc = net_df(11,'Temperature_surface')
ulwrf_sfc = net_df(12,'Upward_Long-Wave_Rad_Flux_surface')
ulwrf_tatm = net_df(13,'Upward_Long-Wave_Rad_Flux')
uswrf_sfc = net_df(14,'Upward_Short-Wave_Rad_Flux')

In [ ]:
station = pd.read_csv(r'C:\Users\MEMORY SISTEMAS\Desktop\Bootcamp\thebridge_ft_sep22 - PB\3-Machine Learning\2-Supervisado\11- Entregables\data\station_info.csv')
station

Se suma toda la columna 'lon' por 360 para que tenga la misma referencia que las variables.

In [ ]:
station['elon'] = station['elon'] +360
station

In [ ]:
# filtro para tablas de varibales para quedarnos con la zona de nuestro panel 'APAC' que dispone de lat 34.9 y lon 261.7 agrupando para tener un dato por día con el que poder hacer el concat
def filter_df_vble(tabla):
    df = (tabla[((tabla.index.get_level_values('lat') > 34) & (tabla.index.get_level_values('lat') < 36)) & ((tabla.index.get_level_values('lon') > 261) & (tabla.index.get_level_values('lon') < 263))])
    df = pd.DataFrame(df)
    df.drop('intValidTime', axis= 1, inplace =  True)
    df = df.groupby('intTime').mean()
    df.reset_index(inplace=True)
    df['intTime'] = df['intTime']/100
    return df

In [ ]:
apcp_sfc_df = filter_df_vble(apcp_sfc)
dlwrf_sfc_df = filter_df_vble(dlwrf_sfc)
dswrf_sfc_df = filter_df_vble(dswrf_sfc)
pres_msl_df = filter_df_vble(pres_msl)
pwat_eatm_df = filter_df_vble(pwat_eatm)
spfh_2m_df = filter_df_vble(spfh_2m)
tcdc_eatm_df = filter_df_vble(tcdc_eatm)
tcolc_eatm_df = filter_df_vble(tcolc_eatm)
tmax_2m_df = filter_df_vble(tmax_2m)
tmin_2m_df = filter_df_vble(tmin_2m)
tmp_2m_df = filter_df_vble(tmp_2m)
tmp_sfc_df = filter_df_vble(tmp_sfc)
ulwrf_sfc_df = filter_df_vble(ulwrf_sfc)
ulwrf_tatm_df = filter_df_vble(ulwrf_tatm)
uswrf_sfc_df = filter_df_vble(uswrf_sfc)


In [ ]:
# Nos traemos lo que será nuestro target
train = pd.read_csv(r'C:\Users\MEMORY SISTEMAS\Desktop\Bootcamp\thebridge_ft_sep22 - PB\3-Machine Learning\2-Supervisado\11- Entregables\data\train.csv')
train

In [ ]:
APAC = train[['Date', 'APAC']]
APAC = APAC.rename(columns={'Date':'intTime'})

In [ ]:
df_final = APAC.merge(apcp_sfc_df, on= 'intTime', how = 'left').merge(dlwrf_sfc_df, on= 'intTime', how = 'left').merge(dswrf_sfc_df, on= 'intTime', how = 'left').merge(pres_msl_df, on= 'intTime', how = 'left').merge(pwat_eatm_df, on= 'intTime', how = 'left').merge(spfh_2m_df, on= 'intTime', how = 'left').merge(tcdc_eatm_df, on= 'intTime', how = 'left').merge(tcolc_eatm_df, on= 'intTime', how = 'left').merge(tmax_2m_df, on= 'intTime', how = 'left').merge(tmin_2m_df, on= 'intTime', how = 'left').merge(tmp_2m_df, on= 'intTime', how = 'left').merge(tmp_sfc_df, on= 'intTime', how = 'left').merge(ulwrf_sfc_df, on= 'intTime', how = 'left').merge(ulwrf_tatm_df, on= 'intTime', how = 'left').merge(uswrf_sfc_df, on= 'intTime', how = 'left')

In [ ]:
from datetime import datetime
df_final[['Date']] = df_final[['intTime']].applymap(str).applymap(lambda s: "{}/{}/{}".format(s[0:4],s[4:6], s[6:]))
df_final['DateTime'] = pd.to_datetime(df_final['Date'])
df_final.drop('intTime', axis = 1, inplace=True)

In [ ]:
def season_of_date(date):
    year = str(date.year)
    seasons = {'spring': pd.date_range(start= year+'/03/21', end= year+'/06/20'),
               'summer': pd.date_range(start=year+'/06/21', end= year +'/09/22'),
               'autumn': pd.date_range(start=year+'/09/23', end= year + '/12/20')}
    if date in seasons['spring']:
        return 'spring'
    if date in seasons['summer']:
        return 'summer'
    if date in seasons['autumn']:
        return 'autumn'
    else:
        return 'winter'

# Assuming df has a date column of type `datetime`
df_final['season'] = df_final.DateTime.map(season_of_date)

In [ ]:
df_final_dummies = pd.get_dummies(df_final['season'], 'season', drop_first= True)

In [ ]:
df_final = pd.concat([df_final, df_final_dummies], axis = 1)
df_final = df_final.drop('season', axis = 1)

In [ ]:
df_final = df_final.drop('DateTime', axis = 1)

In [ ]:
df_final = df_final.set_index('Date')
df_final